<a href="https://colab.research.google.com/github/Talha1818/Chains-in-Langchain/blob/master/Chains_in_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install -q \
langchain \
langchain-core \
langchain-openai \
openai \
langchain-anthropic \
langchain-google-genai \
google-generativeai \
langchain-huggingface \
transformers \
huggingface-hub \
langchain-groq \
python-dotenv \
numpy \
scikit-learn \
grandalf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.3 MB/s eta 0:00:00


# **Simple Chain**
_______________

In [6]:
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
import getpass
import os

In [7]:
prompt = PromptTemplate(
    input_variables=["player"],
    template="Write down the 5 lines about this {player}?",
)

In [8]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········


In [15]:
model = ChatGroq(model="llama3-8b-8192", temperature=0)

In [16]:
parser = StrOutputParser()

In [17]:
chain = prompt | model | parser # make a simple chain

In [21]:
res = chain.invoke({"player": "Virat Kohli"})
print(res)

Here are 5 lines about Virat Kohli:

1. Virat Kohli is an Indian cricketer who is widely regarded as one of the greatest batsmen of all time.
2. He is the current captain of the Indian national team in all formats of the game.
3. Kohli has been a dominant force in international cricket, holding numerous records and awards, including the ICC Cricketer of the Year award.
4. He is known for his aggressive batting style, which has earned him the nickname "The Run Machine".
5. Kohli has also been a successful captain, leading India to several victories in Test matches, One-Day Internationals (ODIs), and Twenty20 Internationals (T20Is).


In [24]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +----------+         
      | ChatGroq |         
      +----------+         
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


# **Sequential Chain**
____________

In [25]:
prompt1 = PromptTemplate(
    input_variables=["player"],
    template="Write details report on this {player}?",
)

prompt2 = PromptTemplate(
    input_variables=['player_details'],
    template="Write down the 5 lines about this {player_details}?",
)

In [26]:
chain = prompt1 | model | parser | prompt2 | model | parser

In [27]:
print(chain.invoke({"player": "Virat Kohli"}))

Here are the 5 lines about Virat Kohli:

1. Virat Kohli was born on November 5, 1988, in Delhi, India, and began playing cricket at a young age.
2. He made his One-Day International (ODI) debut for India in 2008 against Sri Lanka and scored his first ODI century in 2011 against West Indies.
3. Kohli has achieved numerous milestones in his career, including being the fastest to 7,000 ODI runs and most ODI centuries by an Indian.
4. He is known for his aggressive and dominant style of play, particularly effective against fast bowling, and has an average of 58.21 in ODI cricket, the highest among all batsmen with a minimum of 5,000 runs.
5. Kohli has been the captain of the Indian Test team since 2015 and has led the team to several victories, including a series win in Australia in 2018-19, and is known for his tactical acumen and ability to motivate his teammates.


In [28]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +----------+         
      | ChatGroq |         
      +----------+         
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *       

# **Parallel Chain**
__________

In [29]:
prompt1 = PromptTemplate(
    template='Generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}',
    input_variables=['notes', 'quiz']
)

In [30]:
from langchain.schema.runnable import RunnableParallel

In [31]:
parallel_chain = RunnableParallel({
    "notes": prompt1 | model | parser,
    "quiz": prompt2 | model | parser
})

In [32]:
merge_chain = prompt3 | model | parser

In [33]:
chain = parallel_chain | merge_chain

In [34]:
text = """
Virat Kohli (born 5 November 1988)[b] is an Indian international cricketer who plays Test and ODI cricket for the national team and is a former captain in all formats.[5] He is a right-handed batsman and an occasional right arm medium pace bowler. He is called the king, the chase master and the run machine for his playing style, records and ability to lead the team to victory.[6] Kohli is the highest run-scorer in the Indian Premier League, third in T20I, third in ODI, and third in international cricket.[7] Regarded as one of the greatest batsmen of all time, he holds the record for the most ODI centuries and has the second most centuries in international cricket.[8] Kohli is also the only captain to lead India to three consecutive Test mace wins during 2017, 2018, and 2019.[9]

Kohli led the Indian team to victory at the 2008 U19 World Cup and was a key member of the teams that won the 2011 ODI World Cup, 2013 Champions Trophy, 2024 T20 World Cup and 2025 Champions Trophy. He represents Royal Challengers Bengaluru in the Indian Premier League and Delhi in domestic cricket. In 2013, Kohli was ranked number one in the ODI batting rankings. In 2015, he achieved the same in T20I.[10] In 2018, he was ranked the number one Test batsman, making him the only Indian to hold the number one spot in all three formats. He is the first player to score 20,000 runs in a decade. ICC named him the male cricketer of the decade for the years 2011 to 2020.[11]

Kohli has garnered 10 ICC Awards making him the most awarded player in international cricket history. He won the ODI Player of the Year award four times in 2012, 2017, 2018, and 2023. He has also won the Sir Garfield Sobers Trophy given to the Cricketer of the Year, on two occasions, in 2017 and 2018. In 2018, he touched his peak and became the first player to win all the three major awards Garfield Sobers trophy, ODI and Test Player of the Year awards in the same year. He was awarded with Spirit of Cricket Award in 2019. He was also awarded with Cricketer of the Decade and ODI Cricketer of the Decade in 2020. Kohli was named the Wisden Leading Cricketer in the World for three consecutive years, from 2016 to 2018. Kohli has the most Player of the Series awards and second most Player of the Match awards to his name in all three formats combined. He was honoured with the Arjuna Award in 2013, the Padma Shri in 2017, and India's highest sporting honour, the Khel Ratna Award, in 2018. Time magazine included him on its list of the 100 most influential people in the world in 2018.
"""


In [36]:
print(chain.invoke({"text": text}))

Here is the merged document:

**Virat Kohli**

* Born: November 5, 1988
* Indian international cricketer
* Right-handed batsman and occasional right-arm medium pace bowler
* Known as "King", "Chase Master", and "Run Machine"
* Highest run-scorer in IPL, third in T20I, ODI, and international cricket
* Regarded as one of the greatest batsmen of all time
* Holds records for most ODI centuries and second most centuries in international cricket

**Achievements**

* Led India to three consecutive Test mace wins (2017, 2018, 2019)
* Won ICC Awards 10 times, most awarded player in international cricket history
* Won ODI Player of the Year award four times (2012, 2017, 2018, 2023)
* Won Sir Garfield Sobers Trophy (Cricketer of the Year) twice (2017, 2018)
* First player to win all three major awards in the same year (2018)
* Named Wisden Leading Cricketer in the World three times (2016-2018)
* Honoured with Arjuna Award, Padma Shri, Khel Ratna Award, and Time magazine's 100 most influential peo

In [37]:
chain.get_graph().print_ascii()

          +---------------------------+            
          | Parallel<notes,quiz>Input |            
          +---------------------------+            
                ***             ***                
              **                   **              
            **                       **            
+----------------+              +----------------+ 
| PromptTemplate |              | PromptTemplate | 
+----------------+              +----------------+ 
          *                             *          
          *                             *          
          *                             *          
    +----------+                  +----------+     
    | ChatGroq |                  | ChatGroq |     
    +----------+                  +----------+     
          *                             *          
          *                             *          
          *                             *          
+-----------------+            +-----------------+ 
| StrOutputP

# **Conditional Chain**
________________

In [38]:
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal

In [39]:
class Feedback(BaseModel):
    sentiment: Literal["positive", "neutral", "negative"] = Field(description="sentiment of text")
    language: str = Field(description="language of text")

In [40]:
parser2 = PydanticOutputParser(pydantic_object=Feedback)

In [41]:
prompt1 = PromptTemplate(
    template='Classify the sentiment (positive, negative, neutral) and language of this feedback \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={"format_instruction": parser2.get_format_instructions()}
)

In [42]:
classifier_chain = prompt1 | model | parser2

In [45]:
res = classifier_chain.invoke({"feedback": "This is a great product"})
print(res)

sentiment='positive' language='English'


In [54]:
prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

prompt4 = PromptTemplate(
    template='Write an appropriate response to this neutral feedback \n {feedback}',
    input_variables=['feedback']
)

In [58]:
branch_chain = RunnableBranch(
    (lambda x: x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x: x.sentiment == 'negative', prompt3 | model | parser),
    (lambda x: x.sentiment == 'neutral', prompt4 | model | parser),
    RunnableLambda(lambda x: "could not find sentiment & language")
)

In [59]:
chain = classifier_chain | branch_chain

In [60]:
res = chain.invoke({"feedback": "This is a great product"})
print(res)

Here's a potential response to positive feedback:

"Wow, thank you so much for your kind words! We're thrilled to hear that you've had a great experience with us. Your feedback means a lot to us, and we're glad to know that we've been able to meet your expectations. If there's anything else we can do to make your experience even better, please don't hesitate to let us know. Thanks again for your feedback - we appreciate it!"

This response acknowledges the positive feedback, expresses gratitude, and shows appreciation for the customer's input. It also leaves the door open for further communication and feedback, which can help to build a stronger relationship with the customer.


In [62]:
res = chain.invoke({"feedback": "This is not bad but we want excellent product quality"})
print(res)

Here's a potential response to neutral feedback:

"Thank you for taking the time to share your thoughts with us. We appreciate your feedback and will take it into consideration as we continue to improve our [product/service]. If you have any specific suggestions or ideas on how we can better meet your needs, please don't hesitate to let us know. We're always looking for ways to enhance our offerings and provide a better experience for our customers."

This response acknowledges the customer's feedback, expresses gratitude for their input, and invites them to provide more specific suggestions or ideas. It also maintains a professional tone and doesn't make any promises or commitments that may be difficult to fulfill.


In [63]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     
